## SQL 언어 활용을 위한 라이브러리, 쿼리 작성

In [4]:
import pandas as pd
from pandasql import *

## db의 테이블이라고 가정할 csv

In [3]:
df = pd.read_csv('songhunhwa.github.com\\tutorial\\tutorial_01\\doc_use_log.csv'

In [6]:
q = "SELECT * FROM df WHERE ext = 'PDF' AND  ismydoc = '0' LIMIT 10"

In [7]:
print(sqldf(q, locals()).to_string())

  actiontype  ismydoc  ext                         sessionid documentposition   datetime
0       OPEN        0  PDF  9400fd2e43d7dc2d054ca78806236ee1     LOCALSTORAGE  2016.7.18
1      CLOSE        0  PDF  9400fd2e43d7dc2d054ca78806236ee1     LOCALSTORAGE  2016.7.18
2       OPEN        0  PDF  f191063c562691041dfa935ff0876975         OTHERAPP   2016.7.6
3      CLOSE        0  PDF  3747719d8f6bdd334703320433928d5a         OTHERAPP  2016.7.20
4       OPEN        0  PDF  3747719d8f6bdd334703320433928d5a         OTHERAPP  2016.7.20
5       OPEN        0  PDF  3da5ab986c93803de1e25012d9972274         OTHERAPP  2016.7.28
6      CLOSE        0  PDF  3da5ab986c93803de1e25012d9972274         OTHERAPP  2016.7.28
7       OPEN        0  PDF  9e37751e132b5eb96e7d3fde7db132e3         OTHERAPP  2016.7.19
8      CLOSE        0  PDF  9e37751e132b5eb96e7d3fde7db132e3         OTHERAPP  2016.7.19
9       OPEN        0  PDF  202c847b75c9920eb51337b1d5181e10         OTHERAPP  2016.7.24


## SQL grouop by 함수를 이용한 집계 처리    

In [8]:
q = """
    SELECT
        ext,
        count(ext) as count,
        count(distinct sessionid) as unq_sess
    FROM df
    GROUP BY ext
    ORDER BY count DESC
"""

In [9]:
print(sqldf(q, locals()).to_string())

      ext  count  unq_sess
0     PDF  82004     37404
1    DOCX  58303     24842
2    XLSX  52118     23448
3     HWP  26244     11266
4     DOC  24586     11174
5     XLS  24489     11208
6    PPTX  15385      7053
7     TXT   9814      4569
8     PPT   5382      2596
9    PPSX   2374      1212
10    ODT    820       375
11    PPS    324       153
12    JPG      9         9
13  SHEET      5         3
14   WORD      2         1
15    PNG      2         2


## 테이블 조인

In [11]:
ios = pd.read_csv('songhunhwa.github.com\\tutorial\\tutorial_01\\ios.csv')

In [12]:
q = """
    SELECT
        A.*,
        B.flag
    FROM df A
    LEFT JOIN
        (
        SELECT sessionid, flag
        FROM ios
        ) B
        ON A.sessionid = B.sessionid
    WHERE B.flag = 'iOS'
"""

In [13]:
print(sqldf(q, locals()).to_string())

    actiontype  ismydoc   ext                         sessionid documentposition   datetime flag
0        CLOSE        1   PDF  b8a35b63d65e36fcbcf7284db362fa06   MYPOLARISDRIVE  2016.7.29  iOS
1         OPEN        1   PDF  b8a35b63d65e36fcbcf7284db362fa06   MYPOLARISDRIVE  2016.7.29  iOS
2        CLOSE        0   PDF  ab9ea758f63a61463efa059c7183b976         OTHERAPP  2016.7.29  iOS
3         OPEN        0   PDF  ab9ea758f63a61463efa059c7183b976         OTHERAPP  2016.7.29  iOS
4        CLOSE        0   PDF  401e95152f65bcd9573bb1707967e82a         OTHERAPP  2016.7.17  iOS
5         OPEN        0   PDF  401e95152f65bcd9573bb1707967e82a         OTHERAPP  2016.7.17  iOS
6        CLOSE        0   PDF  7891c7016850ccca28b54b3f8f0c554e         OTHERAPP  2016.7.25  iOS
7         OPEN        0   PDF  7891c7016850ccca28b54b3f8f0c554e         OTHERAPP  2016.7.25  iOS
8        CLOSE        1   HWP  dd16321402b19a3f20261e6905e4e9bf         OTHERAPP  2016.7.12  iOS
9         OPEN        1   HWP 

## 문제1
## df 테이블의 Action Type 값(항목)별 유니크한 세션수는?<br>유니크 세션수 기준으로 내림차순 정렬하기

In [39]:
q = """
    SELECT
        actiontype,
        count(distinct sessionid) as session_cnt
    FROM df
    GROUP BY actiontype
    ORDER BY session_cnt DESC
"""

In [42]:
print(sqldf(q, locals()).to_string())

     actiontype  session_cnt_uniq
0          OPEN            112771
1         CLOSE             95901
2          SAVE              8251
3         RESET              7467
4        SAVEAS              4488
5   EXPORT_SAME                57
6        EXPORT                17
7  SAVEAS_OTHER                 5


In [43]:
df.groupby('actiontype')['sessionid'].nunique().sort_values(ascending = False).reset_index()

,actiontype,sessionid
0,OPEN,112771
1,CLOSE,95901
2,SAVE,8251
3,RESET,7467
4,SAVEAS,4488
5,EXPORT_SAME,57
6,EXPORT,17
7,SAVEAS_OTHER,5


## 문제2
## ismydoc이 1(True)인 경우에 한해, 날짜별 세션수의 유니크 빈도 구하기<br>유니크 빈도가 가장 큰 top 5 날짜 확인하기

In [47]:
q = """
    SELECT
        datetime,
        count(distinct sessionid) as unq_date
    FROM df
    WHERE ismydoc = True
    GROUP BY datetime
    ORDER BY unq_date DESC
    LIMIT 5
"""

In [48]:
print(sqldf(q, locals()).to_string())

    datetime  unq_date
0  2016.7.12      2425
1  2016.7.19      2325
2   2016.7.1      2297
3  2016.7.13      2294
4  2016.7.14      2286


In [50]:
df.query('ismydoc==True').groupby('datetime')['sessionid'].nunique().sort_values(ascending = False)[:5].reset_index()

,datetime,sessionid
0,2016.7.12,2425
1,2016.7.19,2325
2,2016.7.1,2297
3,2016.7.13,2294
4,2016.7.14,2286


## 문제3
## 문서 포지션별(documentposition)로 자주 OPEN 되는 확장자(ext)를 확인하기<br>카운트 기준: unique sessionid<br>그룹별, 세션카운트 기준 desc 정렬

In [63]:
q = """
    SELECT
        documentposition
        ext,
        count(distinct sessionid) as unq_session
    FROM df
    WHERE actiontype = "OPEN"
    GROUP BY documentposition, ext
    ORDER BY documentposition, unq_session DESC
"""

In [64]:
print(sqldf(q, locals()).to_string())

               ext  unq_session
0     LOCALSTORAGE         1591
1     LOCALSTORAGE         1531
2     LOCALSTORAGE         1465
3     LOCALSTORAGE         1458
4     LOCALSTORAGE          620
5     LOCALSTORAGE          617
6     LOCALSTORAGE          564
7     LOCALSTORAGE          450
8     LOCALSTORAGE          158
9     LOCALSTORAGE           10
10    LOCALSTORAGE            9
11    LOCALSTORAGE            3
12  MYPOLARISDRIVE         5675
13  MYPOLARISDRIVE         4924
14  MYPOLARISDRIVE         4742
15  MYPOLARISDRIVE         2008
16  MYPOLARISDRIVE         1968
17  MYPOLARISDRIVE         1408
18  MYPOLARISDRIVE         1404
19  MYPOLARISDRIVE         1161
20  MYPOLARISDRIVE         1147
21  MYPOLARISDRIVE          361
22  MYPOLARISDRIVE           44
23  MYPOLARISDRIVE            3
24  MYPOLARISDRIVE            1
25     NEWDOCUMENT         2244
26     NEWDOCUMENT          715
27     NEWDOCUMENT          616
28     NEWDOCUMENT          502
29     NEWDOCUMENT          123
30     N

In [79]:
df.query("actiontype=='OPEN'").groupby(['documentposition', 
                                       'ext'])['sessionid'].count().unstack().fillna(0)

ext,DOC,DOCX,HWP,ODT,PDF,PPS,PPSX,PPT,PPTX,SHEET,TXT,WORD,XLS,XLSX
documentposition,,,,,,,,,,,,,,
LOCALSTORAGE,454.0,1472.0,1598.0,10.0,1467.0,9.0,3.0,158.0,622.0,0.0,572.0,0.0,618.0,1537.0
MYPOLARISDRIVE,1457.0,4927.0,2067.0,47.0,5117.0,0.0,1149.0,372.0,1223.0,3.0,1443.0,1.0,2029.0,5798.0
NEWDOCUMENT,5.0,2253.0,623.0,0.0,1.0,0.0,0.0,3.0,504.0,0.0,124.0,0.0,19.0,720.0
OTHERAPP,10407.0,19794.0,8738.0,363.0,35945.0,145.0,51.0,2280.0,5531.0,0.0,2584.0,0.0,9630.0,17224.0
OTHERCLOUD,23.0,82.0,28.0,0.0,96.0,0.0,0.0,8.0,23.0,0.0,16.0,0.0,47.0,329.0
SHAREDDOCUMENT,0.0,16.0,15.0,0.0,9.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,1.0,4.0
